In [12]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical   # For one hot encoding

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
# Feature scaling
x_train = x_train/255.0
x_test = x_test/255.0

In [15]:
# One hot encoder to labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [16]:
model = Sequential([
    # Flatten(input_shape=(28, 28)),
    Input(shape=(28, 28)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])

2025-09-10 14:51:55.968412: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [19]:
model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ['accuracy'])

In [20]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9280 - loss: 0.2560
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9667 - loss: 0.1122
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9763 - loss: 0.0784
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9819 - loss: 0.0590
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9863 - loss: 0.0451


In [22]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"The test accuracy is {test_acc}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9770 - loss: 0.0734   
The test accuracy is 0.9769999980926514
